#libraries and pips

In [ ]:
!pip install ultralytics

In [2]:
from ultralytics import YOLO

c:\Users\falla\miniconda3\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


#Preparing data

In [3]:
import os
import requests
import xml.etree.ElementTree as ET
import yaml
from tqdm import tqdm
from pathlib import Path
import zipfile

def download_file(url, destination):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024
    t = tqdm(total=total_size, unit='iB', unit_scale=True)

    with open(destination, 'wb') as file:
        for data in response.iter_content(block_size):
            t.update(len(data))
            file.write(data)

    t.close()
    if total_size != 0 and t.n != total_size:
        print("Error downloading the file.")

def download_images(urls, destination):
    Path(destination).mkdir(parents=True, exist_ok=True)
    for url in urls:
        filename = os.path.join(destination, os.path.basename(url))
        download_file(url, filename)

def extract_zip(zip_file, destination):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(destination)

def convert_label(path, lb_path, year, image_id, class_names):
    def convert_box(size, box):
        dw, dh = 1. / size[0], 1. / size[1]
        x, y, w, h = (box[0] + box[1]) / 2.0 - 1, (box[2] + box[3]) / 2.0 - 1, box[1] - box[0], box[3] - box[2]
        return x * dw, y * dh, w * dw, h * dh

    in_file = open(path /'images'/'VOCdevkit/' f'VOC{year}/Annotations/{image_id}.xml')
    out_file = open(lb_path, 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        cls = obj.find('name').text
        if cls in class_names and int(obj.find('difficult').text) != 1:
            xmlbox = obj.find('bndbox')
            bb = convert_box((w, h), [float(xmlbox.find(x).text) for x in ('xmin', 'xmax', 'ymin', 'ymax')])
            cls_id = class_names.index(cls)  # class id
            out_file.write(" ".join(str(a) for a in (cls_id, *bb)) + '\n')

if __name__ == "__main__":
    # Specify the download URLs
    urls = [
        'https://github.com/ultralytics/yolov5/releases/download/v1.0/VOCtrainval_06-Nov-2007.zip',
        'https://github.com/ultralytics/yolov5/releases/download/v1.0/VOCtest_06-Nov-2007.zip',
        'https://github.com/ultralytics/yolov5/releases/download/v1.0/VOCtrainval_11-May-2012.zip'
    ]

    # Specify the destination directory
    destination_dir = '/content/datasets/VOC/images'

    # Download the files
    download_images(urls, destination_dir)

    # Specify the destination directory for extracted files
    extract_dir = '/content/datasets/VOC/images/'

    # Extract the downloaded zip files
    for url in urls:
        zip_file = os.path.join(destination_dir, os.path.basename(url))
        extract_zip(zip_file, extract_dir)

    # Define class names
    class_names = [
        'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat',
        'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person',
        'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'
    ]




100%|██████████| 446M/446M [20:02<00:00, 371kiB/s]    
 45%|████▌     | 197M/438M [14:27<17:37, 228kiB/s]     


Error downloading the file.


  0%|          | 841k/1.95G [03:05<119:41:05, 4.52kiB/s] 


Error downloading the file.


BadZipFile: File is not a zip file

In [4]:
from pathlib import Path
from tqdm import tqdm
import os
class_names = [
        'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat',
        'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person',
        'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'
    ]
# Define the parent directory
dir = Path('./datasets/VOC')  # Specify the parent directory

# Iterate over each year and image set
for year, image_set in [('2012', 'train'), ('2012', 'val')]:
    # Define paths for images and labels
    imgs_path = dir / 'images' / f'{image_set}{year}'
    lbs_path = dir / 'labels' / f'{image_set}{year}'
    imgs_path.mkdir(exist_ok=True, parents=True)  # Create images directory if not exists
    lbs_path.mkdir(exist_ok=True, parents=True)   # Create labels directory if not exists

    # Read image IDs from the text file
    with open(dir / 'images' / 'VOCdevkit' / f'VOC{year}' / 'ImageSets' / 'Main' / f'{image_set}.txt') as f:
        image_ids = f.read().strip().split()

    # Iterate over each image ID
    for id in tqdm(image_ids, desc=f'{image_set}{year}'):
        # Define old image path and new label path
        img_old_path = dir / 'images' / 'VOCdevkit' / f'VOC{year}' / 'JPEGImages' / f'{id}.jpg'
        lb_new_path = (lbs_path / f'{id}.txt').with_suffix('.txt')

        # Check if the image file exists
        if img_old_path.exists():
            # Move image to the new directory
            os.rename(img_old_path, imgs_path / f'{id}.jpg')

            # Convert labels to YOLO format
            convert_label(dir, lb_new_path, year, id , class_names)
        else:
            print(f"Image file '{img_old_path}' not found. Skipping...")


val2012: 100%|██████████| 5823/5823 [00:02<00:00, 2853.19it/s]


In [5]:
!mkdir -p /content/datasets/VOC_l/images/train2012
!mkdir -p /content/datasets/VOC_l/labels/train2012


In [6]:
import shutil

source_dir = "/content/datasets/VOC/images/val2012"
destination_dir = "/content/datasets/VOC_l/images/val2012"
shutil.copytree(source_dir, destination_dir)

source_dir = "/content/datasets/VOC/labels/val2012"
destination_dir = "/content/datasets/VOC_l/labels/val2012"
shutil.copytree(source_dir, destination_dir)

'/content/datasets/VOC_l/labels/val2012'

#Augments

In [7]:
import torchvision.transforms.functional as F
import torch.nn.functional as Fun
import random
import numpy as np
import torch
import PIL
from PIL import Image, ImageDraw
import datetime
import os
import time


import random
import torch

from torchvision.transforms import functional as F


def _flip_coco_person_keypoints(kps, width):
    flip_inds = [0, 2, 1, 4, 3, 6, 5, 8, 7, 10, 9, 12, 11, 14, 13, 16, 15]
    flipped_data = kps[:, flip_inds]
    flipped_data[..., 0] = width - flipped_data[..., 0]
    # Maintain COCO convention that if visibility == 0, then x, y = 0
    inds = flipped_data[..., 2] == 0
    flipped_data[inds] = 0
    return flipped_data


class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target


class RandomHorizontalFlip(object):
    def __init__(self, prob):
        self.prob = prob

    def __call__(self, image, target):
        if random.random() < self.prob:
            height, width = image.shape[-2:]
            image = image.flip(-1)
            bbox = target["boxes"]
            bbox[:, [0, 2]] = width - bbox[:, [2, 0]]
            target["boxes"] = bbox
            if "masks" in target:
                target["masks"] = target["masks"].flip(-1)
            if "keypoints" in target:
                keypoints = target["keypoints"]
                keypoints = _flip_coco_person_keypoints(keypoints, width)
                target["keypoints"] = keypoints
        return image, target


class ToTensor(object):
    def __call__(self, image, target):
        image = F.to_tensor(image)
        return image, target


def HorizontalFlipFeatures(image, features):
    image = F.to_tensor(image)
    image = image.flip(-1)
    new_features = {}
    for k in features:
        new_features[k] = features[k].detach().flip(-1)
    return image, new_features


def HorizontalFlip(image, bbox):
    import torchvision.transforms.functional as F
    from PIL import Image

    if isinstance(image, Image.Image):
        image = F.to_tensor(image)
    height, width = image.shape[-2:]
    image = image.flip(-1)
    b = bbox.clone()
    b[:, [0, 2]] = width - bbox[:, [2, 0]]

    # Convert the image back to PIL.Image.Image
    image = F.to_pil_image(image)

    return image, b


def resizeFeatures(img, features, ratio):
    if not type(img) == PIL.Image.Image:
        img = F.to_pil_image(img)
    w, h = img.size
    iw = int(w * ratio)
    ih = int(h * ratio)
    new_features = {}
    for k in features:
        fw = int(features[k].shape(-2) * ratio)
        fh = int(features[k].shape(-1) * ratio)
        new_features[k] = Fun.interpolate(features[k].detach(), (fw, fh))
    return F.to_tensor(img.resize((iw, ih), Image.BILINEAR)), new_features


def resize(image, boxes, ratio):
    if isinstance(image, Image.Image):
        image = F.to_tensor(image)
    h = image.size(1)
    w = image.size(2)
    # w, h = image.size
    ow = int(w * ratio)
    oh = int(h * ratio)

    image = F.to_pil_image(image)
    image = F.to_tensor(image.resize((ow, oh), Image.BILINEAR))
    image = F.to_pil_image(image)

    return image, boxes * ratio


def ColorSwap(image):
    perms = ((0, 1, 2), (0, 2, 1), (1, 0, 2),
             (1, 2, 0), (2, 0, 1), (2, 1, 0))
    swap = perms[random.randint(0, len(perms) - 1)]
    image = F.to_tensor(image)
    image = image[swap, :, :]
    return image


def ColorAdjust(image, factor):
    image = F.adjust_brightness(image, factor)
    image = F.adjust_contrast(image, factor)
    image = F.adjust_saturation(image, factor)
    return F.to_tensor(image)


def GaussianNoise(image, std=1):
    image = F.to_tensor(image)
    x = image + torch.randn(image.size()) * std / 255.0
    return x


def SaltPepperNoise(image, prob):
    image = F.to_tensor(image)
    noise = torch.rand(image.size())
    salt = torch.max(image)
    pepper = torch.min(image)
    image[noise < prob / 2] = salt
    image[noise > 1 - prob / 2] = pepper
    return image


def cutout(image, boxes, cut_num=2, fill_val=0, bbox_remove_thres=0.4, bbox_min_thres=0.1):
    '''
        Cutout augmentation
        image: A PIL image
        boxes: bounding boxes, a tensor of dimensions (#objects, 4)
        labels: labels of object, a tensor of dimensions (#objects)
        fill_val: Value filled in cut out
        bbox_remove_thres: Threshold to remove bbox cut by cutout

        Out: new image, new_boxes, new_labels
    '''
    # Convert PIL image to tensor if it's not already
    if isinstance(image, Image.Image):
        image = F.to_tensor(image)

    # Ensure image tensor is on the same device as other tensors
    device = boxes.device
    image = image.to(device)

    original_h = image.size(1)
    original_w = image.size(2)
    original_channel = image.size(0)

    count = 0
    for _ in range(50):
        # Random cutout size: [0.15, 0.5] of original dimension
        cutout_size_h = random.uniform(0.05 * original_h, 0.2 * original_h)
        cutout_size_w = random.uniform(0.05 * original_w, 0.2 * original_w)

        # Random position for cutout
        left = random.uniform(0, original_w - cutout_size_w)
        right = left + cutout_size_w
        top = random.uniform(0, original_h - cutout_size_h)
        bottom = top + cutout_size_h
        cutout = torch.FloatTensor([int(left), int(top), int(right), int(bottom)]).to(device)

        # Calculate intersect between cutout and bounding boxes
        overlap_size = intersect(cutout.unsqueeze(0), boxes)
        area_boxes = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])
        ratio = overlap_size / area_boxes
        # If all boxes have IoU greater than bbox_remove_thres, try again
        if ratio.max().item() > bbox_remove_thres or ratio.max().item() < bbox_min_thres:
            continue

        cutout_arr = torch.full((original_channel, int(bottom) - int(top), int(right) - int(left)), fill_val).to(device)
        image[:, int(top):int(bottom), int(left):int(right)] = cutout_arr
        count += 1
        if count >= cut_num:
            break

    # Convert tensor back to PIL image
    image = F.to_pil_image(image.cpu())

    return image



import torch
import torchvision.transforms.functional as F
import numpy as np
from PIL import Image

import torch
import torchvision.transforms.functional as F
import numpy as np
from PIL import Image

import torch
import torchvision.transforms.functional as F
import numpy as np
from PIL import Image

def rotate(image, boxes, angle, device='cpu'):
    '''
        Rotate image and bounding box
        image: A PIL image (w, h)
        boxes: A tensors of dimensions (#objects, 4)
        device: Device to use for the tensors ('cpu' or 'cuda')

        Out: rotated image (w, h), rotated boxes
    '''
    if isinstance(image, Image.Image):
        image = F.to_tensor(image).to(device)
    if not isinstance(image, Image.Image):
        image = F.to_pil_image(image)
    new_image = image.copy()
    new_boxes = boxes.clone().to(device)

    # Rotate image, expand = True
    w = image.width
    h = image.height
    cx = w / 2
    cy = h / 2
    new_image = new_image.rotate(angle, expand=True)
    angle = np.radians(angle)
    alpha = np.cos(angle)
    beta = np.sin(angle)
    # Get affine matrix
    AffineMatrix = torch.tensor([[alpha, beta, (1 - alpha) * cx - beta * cy],
                                 [-beta, alpha, beta * cx + (1 - alpha) * cy]], device=device)

    # Rotation boxes
    box_width = (boxes[:, 2] - boxes[:, 0]).reshape(-1, 1)
    box_height = (boxes[:, 3] - boxes[:, 1]).reshape(-1, 1)

    # Get corners for boxes
    x1 = boxes[:, 0].reshape(-1, 1)
    y1 = boxes[:, 1].reshape(-1, 1)

    x2 = x1 + box_width
    y2 = y1

    x3 = x1
    y3 = y1 + box_height

    x4 = boxes[:, 2].reshape(-1, 1)
    y4 = boxes[:, 3].reshape(-1, 1)

    corners = torch.stack((x1, y1, x2, y2, x3, y3, x4, y4), dim=1)
    corners = corners.reshape(-1, 2)  # Tensors of dimension (4* #objects, 2)
    corners = corners.to(device)  # Ensure corners are on the same device as AffineMatrix

    corners = torch.cat((corners, torch.ones(corners.shape[0], 1, device=device)), dim=1)
                        # (Tensors of dimension (4* #objects, 3))

    # Ensure AffineMatrix and corners are on the same device
    AffineMatrix = AffineMatrix.to(device)

    cos = np.abs(AffineMatrix[0, 0].cpu())
    sin = np.abs(AffineMatrix[0, 1].cpu())

    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
    AffineMatrix[0, 2] += (nW / 2) - cx
    AffineMatrix[1, 2] += (nH / 2) - cy

    # Apply affine transform
    rotate_corners = torch.mm(AffineMatrix.float(), corners.t()).t()
    rotate_corners = rotate_corners.reshape(-1, 8)

    x_corners = rotate_corners[:, [0, 2, 4, 6]]
    y_corners = rotate_corners[:, [1, 3, 5, 7]]

    # Get (x_min, y_min, x_max, y_max)
    x_min, _ = torch.min(x_corners, dim=1)
    x_min = x_min.reshape(-1, 1)
    y_min, _ = torch.min(y_corners, dim=1)
    y_min = y_min.reshape(-1, 1)
    x_max, _ = torch.max(x_corners, dim=1)
    x_max = x_max.reshape(-1, 1)
    y_max, _ = torch.max(y_corners, dim=1)
    y_max = y_max.reshape(-1, 1)

    new_boxes = torch.cat((x_min, y_min, x_max, y_max), dim=1)

    scale_x = new_image.width / w
    scale_y = new_image.height / h

    # Resize new image to (w, h)
    new_image = new_image.resize((w, h))

    # Resize boxes
    new_boxes /= torch.Tensor([scale_x, scale_y, scale_x, scale_y])
    new_boxes[:, 0] = torch.clamp(new_boxes[:, 0], 0, w)
    new_boxes[:, 1] = torch.clamp(new_boxes[:, 1], 0, h)
    new_boxes[:, 2] = torch.clamp(new_boxes[:, 2], 0, w)
    new_boxes[:, 3] = torch.clamp(new_boxes[:, 3], 0, h)

    image = F.to_tensor(new_image).to(device)
    image = F.to_pil_image(image)
    return image, new_boxes




def intersect(boxes1, boxes2):
    '''
        Find intersection of every box combination between two sets of box
        boxes1: bounding boxes 1, a tensor of dimensions (n1, 4)
        boxes2: bounding boxes 2, a tensor of dimensions (n2, 4)

        Out: Intersection each of boxes1 with respect to each of boxes2,
             a tensor of dimensions (n1, n2)
    '''
    device = boxes1.device  # Get the device of boxes1
    n1 = boxes1.size(0)
    n2 = boxes2.size(0)

    # Ensure both tensors are on the same device
    boxes1 = boxes1.to(device)
    boxes2 = boxes2.to(device)

    max_xy = torch.min(boxes1[:, 2:].unsqueeze(1).expand(n1, n2, 2),
                       boxes2[:, 2:].unsqueeze(0).expand(n1, n2, 2))

    min_xy = torch.max(boxes1[:, :2].unsqueeze(1).expand(n1, n2, 2),
                       boxes2[:, :2].unsqueeze(0).expand(n1, n2, 2))
    inter = torch.clamp(max_xy - min_xy, min=0)  # (n1, n2, 2)
    return inter[:, :, 0] * inter[:, :, 1]  # (n1, n2)


# def draw_PIL_image(image, boxes, labels, name, no=None):
#     '''
#         Draw PIL image
#         image: A PIL image
#         labels: A tensor of dimensions (#objects,)
#         boxes: A tensor of dimensions (#objects, 4)
#     '''
#     if type(image) != PIL.Image.Image:
#         image = F.to_pil_image(image)
#     new_image = image.copy()
#     labels = labels.tolist()
#     draw = ImageDraw.Draw(new_image)
#     boxes = boxes.tolist()
#     # print(no)
#     if no is not None:
#         for n in no:
#             draw.rectangle(xy=boxes[n], outline='red', width=2)
#     else:
#         for i in range(len(boxes)):
#             draw.rectangle(xy=boxes[i])  # , outline=label_color_map[rev_label_map[labels[i]]])
#     new_image.save('vis/{}.jpg'.format(name))

import matplotlib.pyplot as plt


def draw_PIL_image(image, boxes, labels, scores, name):
    if type(image) != PIL.Image.Image:
        image = F.to_pil_image(image)
    plt.imshow(image)
    plt.axis('off')
    plt.gca().set_axis_off()
    plt.subplots_adjust(top=1, bottom=0, right=1, left=0,
                        hspace=0, wspace=0)
    plt.margins(0, 0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    # for i in range(len(boxes)):
    #     x, y = boxes[i][0], boxes[i][1]
    #     w, h = boxes[i][2] - boxes[i][0], boxes[i][3] - boxes[i][1]
    #     plt.gca().add_patch(
    #         plt.Rectangle((x, y), w, h, fill=False, edgecolor=label_color_map[rev_label_map[labels[i].item()]],
    #                       linewidth=2.5))
    #     # plt.text(x, y, '{}={}'.format(voc_labels[labels[n]], scores[n]), color='color', verticalalignment='bottom',
    #     #              fontsize=4)
    plt.savefig('vis/{}.png'.format(name), dpi=256, bbox_inches='tight', pad_inches=0)
    # plt.show()
    plt.cla()


def draw_PIL_image_1(image, ref_boxes, boxes, ref_labels, labels, scores, pm, name, no=None):
    if type(image) != PIL.Image.Image:
        image = F.to_pil_image(image)
    plt.imshow(image)
    plt.axis('off')
    plt.gca().set_axis_off()
    plt.subplots_adjust(top=1, bottom=0, right=1, left=0,
                        hspace=0, wspace=0)
    plt.margins(0, 0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    i = 0
    if no is not None:
        for n in no:
            if i < 1:
                color = 'green'
                x, y = ref_boxes[n][0], ref_boxes[n][1]
                w, h = ref_boxes[n][2] - ref_boxes[n][0], ref_boxes[n][3] - ref_boxes[n][1]
                plt.text(x, y, '{}={}'.format(voc_labels[ref_labels[n] - 1], round(scores[n].item(), 2)),
                         color='white',
                         verticalalignment='bottom', bbox={'facecolor': color, 'alpha': 1.0},
                         fontsize=24)
            else:
                color = 'red'
                x, y = boxes[n][0], boxes[n][1]
                w, h = boxes[n][2] - boxes[n][0], boxes[n][3] - boxes[n][1]
                plt.text(x, y + h, '{}={}'.format(voc_labels[labels[n] - 1], round(pm[n].item(), 2)), color='white',
                         verticalalignment='bottom', bbox={'facecolor': color, 'alpha': 1.0},
                         fontsize=24)
            i += 1
            plt.gca().add_patch(plt.Rectangle((x, y), w, h, fill=False, edgecolor=color, linewidth=2.5))
    plt.savefig('fig/{}'.format(name), dpi=256, bbox_inches='tight', pad_inches=0)
    # plt.show()
    plt.cla()


def draw_PIL_image_2(image, boxes, ref_labels, name, no=None, color='green'):
    if type(image) != PIL.Image.Image:
        image = F.to_pil_image(image)
    plt.imshow(image)
    plt.axis('off')
    plt.gca().set_axis_off()
    plt.subplots_adjust(top=1, bottom=0, right=1, left=0,
                        hspace=0, wspace=0)
    plt.margins(0, 0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    i = 0
    # if no is not None:
    #     for n in no:
    #         x, y = boxes[n][0], boxes[n][1]
    #         w, h = boxes[n][2] - boxes[n][0], boxes[n][3] - boxes[n][1]
    #         plt.gca().add_patch(plt.Rectangle((x, y), w, h, fill=False, edgecolor=color, linewidth=2.5))
    plt.savefig('fig/o_{}.eps'.format(name), dpi=256, bbox_inches='tight', pad_inches=0)
    # plt.show()
    plt.cla()


voc_labels = ('aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car',
              'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
              'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor')
label_map = {k: v + 1 for v, k in enumerate(voc_labels)}
# Inverse mapping
rev_label_map = {v: k for k, v in label_map.items()}
# Colormap for bounding box
CLASSES = 20
distinct_colors = ["#" + ''.join([random.choice('0123456789ABCDEF') for j in range(6)])
                   for i in range(CLASSES)]
label_color_map = {k: distinct_colors[i] for i, k in enumerate(label_map.keys())}

#Active learning

In [8]:
import tensorflow as tf
import numpy as np
from PIL import Image

def calculate_iou(box1, box2):
    # Calculate the coordinates of the intersection rectangle
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # If the intersection is negative (no intersection), return 0
    if x2 < x1 or y2 < y1:
        return 0.0

    # Calculate the area of intersection rectangle
    intersection_area = (x2 - x1) * (y2 - y1)

    # Calculate the area of both input rectangles
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    # Calculate the Union area by adding areas of both boxes and subtracting the intersection area
    union_area = box1_area + box2_area - intersection_area

    # Calculate the Intersection over Union (IoU) by dividing the intersection area by the union area
    iou = intersection_area / union_area

    return iou

In [9]:
import tensorflow as tf
import numpy as np
from PIL import Image
from scipy.stats import entropy

# Precompute the augmented images and boxes once outside the loop
def precompute_augmented_images(image_path, original_image_boxes):
    flip_image, flip_boxes = HorizontalFlip(image_path, original_image_boxes)
    cutout_image = cutout(image_path, original_image_boxes, 2)
    resize_image, resize_boxes = resize(image_path, original_image_boxes, 0.8)
    rot_image, rot_boxes = rotate(image_path, original_image_boxes, 5)

    return [flip_image, cutout_image, resize_image, rot_image], [flip_boxes, original_image_boxes, resize_boxes, rot_boxes]

def get_uncertainty(model, image_path):
    consistency1 = 0
    consistency2 = 0

    original_image_results = model(image_path, verbose=False)
    if len(original_image_results[0].boxes) == 0:
        return 2

    original_image_confs = original_image_results[0].boxes.conf
    original_image_boxes = original_image_results[0].boxes.xyxy
    original_image_labels = original_image_results[0].boxes.cls

    augs = ['flip', 'cut_out', 'smaller_resize', 'rotation']

    # Precompute augmented images and boxes
    augmented_images, augmented_boxes = precompute_augmented_images(image_path, original_image_boxes)

    for aug_image, aug_boxes, aug_name in zip(augmented_images, augmented_boxes, augs):
        aug_image_results = model(aug_image, verbose=False)
        aug_image_confs = aug_image_results[0].boxes.conf
        aug_image_boxes = aug_image_results[0].boxes.xyxy

        iou_max = 0
        for orig_box in aug_boxes:
            max_iou = 0
            for aug_box in aug_image_boxes:
                iou = calculate_iou(orig_box, aug_box)
                if iou > max_iou:
                    max_iou = iou
            iou_max += max_iou
        avg_iou_max = iou_max / len(aug_boxes)
        consistency1 += avg_iou_max

    # Classification uncertainty using Jensen-Shannon Divergence
    max_len = max(len(original_image_confs), len(aug_image_confs))
    original_image_confs_padded = np.pad(original_image_confs.cpu().numpy(), (0, max_len - len(original_image_confs)), mode='constant')
    aug_image_confs_padded = np.pad(aug_image_confs.cpu().numpy(), (0, max_len - len(aug_image_confs)), mode='constant')

    # Compute Jensen-Shannon Divergence
    p = (original_image_confs_padded + aug_image_confs_padded) / 2.0
    js_divergence = (entropy(original_image_confs_padded, p) + entropy(aug_image_confs_padded, p)) / 2.0
    consistency2 = 1 - js_divergence

    consistency1 /= len(augs)

    return consistency1 + consistency2


#Training(uncertainty)

In [ ]:
import os
import random
import shutil
import numpy as np

torch.cuda.set_device(0)
random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)

cycles = 7
model = None  # Initialize the model variable
for c in range(cycles):
    # model = YOLO("yolov8n.pt")  # build a new model from YAML


    # Path to the directory containing images and labels for training
    image_dir = "/content/datasets/VOC/images/train2012"
    label_dir = "/content/datasets/VOC/labels/train2012"

    # Path to the directory where you want to move the sampled images and labels
    output_image_dir = "/content/datasets/VOC_l/images/train2012"
    output_label_dir = "/content/datasets/VOC_l/labels/train2012"

    # Number of images to sample
    num_samples = 500

    # Get a list of all image files in the directory
    image_files = os.listdir(image_dir)

    # Sample images randomly for the first cycle
    if c == 0:
        sampled_images = random.sample(image_files, num_samples)
    else:
        # Dictionary to store the uncertainty scores for each image
        uncertainty_scores = {}

        # Calculate uncertainty scores for each image
        for i, image_file in enumerate(image_files):
            image_path = os.path.join(image_dir, image_file)
            image = Image.open(image_path)

            uncertainty_score = get_uncertainty(model, image)

            uncertainty_scores[image_file] = uncertainty_score

            # Print progress every 500 files
            if i % 500 == 0:
                print(f"Processed {i} files out of {len(image_files)}")

        # Sort the images based on their uncertainty scores
        sorted_images = sorted(uncertainty_scores.items(), key=lambda x: x[1])

        # Sample num_samples images with the least uncertainty
        sampled_images = [image_file for image_file, _ in sorted_images[:num_samples]]

    # Move sampled images and their corresponding labels to the output directory
    for image_file in sampled_images:
        # Move image
        image_path = os.path.join(image_dir, image_file)
        shutil.move(image_path, output_image_dir)

        # Extract corresponding label file name
        label_file = image_file.replace(".jpg", ".txt")

        # Move label
        label_path = os.path.join(label_dir, label_file)
        shutil.move(label_path, output_label_dir)

    print("Number of images: ", len([file for file in os.listdir(output_image_dir) if file.endswith(".jpg")]))
    print("Number of label : ", len([file for file in os.listdir(output_label_dir) if file.endswith(".txt")]))

    # Train the model
    model = YOLO("yolov8n.pt")  # build a new model from YAML
    results = model.train(data='/content/VOC_2012.yaml', epochs=20 , batch=4, workers=4,momentum=0.9,weight_decay=0.0001 ,plots=True,optimizer='SGD')


Number of images:  500
Number of label :  500


100%|██████████| 6.23M/6.23M [00:00<00:00, 71.7MB/s]


Ultralytics YOLOv8.1.14 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/VOC_2012.yaml, epochs=20, time=None, patience=50, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=4, project=None, name=train, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=T

100%|██████████| 755k/755k [00:00<00:00, 17.2MB/s]

Overriding model.yaml nc=80 with nc=20

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

...

      13/20      0.82G      1.107      2.323      1.432          7        640: 100%|██████████| 500/500 [01:06<00:00,  7.51it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 728/728 [01:12<00:00, 10.01it/s]


                   all       5823      13841      0.542       0.51      0.517      0.336

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      14/20     0.822G       1.06      2.183      1.382         10        640: 100%|██████████| 500/500 [01:06<00:00,  7.51it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 728/728 [01:13<00:00,  9.92it/s]


                   all       5823      13841      0.571      0.516      0.533      0.351

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      15/20     0.816G      1.034      2.095      1.366          5        640: 100%|██████████| 500/500 [01:08<00:00,  7.28it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|█████████▉| 727/728 [01:12<00:00, 11.76it/s]